In [3]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings
from multiprocessing import Pool
import tensorflow as tf
from tensorflow.python.ops import nn

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Lambda, Dropout
from tensorflow.keras import Input, Model
from utils import *
from functions import * 
from DNN_metrics import * 
from loss_function import * 
from model_DL import *
from metrics import *

py_file_location = "..."
os.path.abspath(os.path.join(os.path.dirname(py_file_location), os.path.pardir))





tf.device('GPU:1') 

2022-03-16 08:58:38.487131: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-16 08:58:46.839040: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-16 08:58:46.856283: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-16 08:58:46.928935: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 72 deviceMemorySize: 44.49GiB deviceMemoryBandwidth: 581.23GiB/s
2022-03-16 08:58:46.929967: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:5e:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 72 deviceMemorySize: 44.49GiB deviceMemoryBandwidth: 581.23GiB/s
2022-03-16 08:5

In [8]:
P1 = 10.0

Primary_ID = [0, 1, 6] 

Secondary_ID = [3, 7]

Nbr_train = int(1E6)

Nbr_test = int(2E5) 

# DNN-parameters :

tau = 0.25

VS = 0.2 # validation_split

Epochs = 500 # Epochs number

BS = 4096 # batch_size

LD = 10**0.5

LR = 10**-4

metrics = [Delta_DNN_CF, Opportunistic_Achievable_Rate_CF(tau),outage_percentage_CF(tau)] #, QoS_mean_DF, QoS_median_DF


callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)


data_path = "Dataset"
  
# Parent Directory path
parent_dir = ""

## Loading the data

In [6]:
### Train ###

dataset_train = np.load(os.path.join(parent_dir,data_path,'dataset_train_GF.npz'))

h_11_tr = dataset_train['h_PP'][:Nbr_train]
h_12_tr = dataset_train['h_PS'][:Nbr_train]
h_1R_tr = dataset_train['h_PR'][:Nbr_train]
h_21_tr = dataset_train['h_SP'][:Nbr_train]
h_22_tr = dataset_train['h_SS'][:Nbr_train]
h_2R_tr = dataset_train['h_SR'][:Nbr_train]
h_R1_tr = dataset_train['h_RP'][:Nbr_train]
h_R2_tr = dataset_train['h_RS'][:Nbr_train]


x_train = np.stack([h_R1_tr, h_11_tr, h_2R_tr, h_1R_tr, h_22_tr, h_R2_tr, h_21_tr, h_12_tr], axis=1)

### Test ###

dataset_test = np.load(os.path.join(parent_dir,data_path,'dataset_val_GF.npz'))

h_11_test = dataset_test['h_PP'][:Nbr_test]
h_12_test = dataset_test['h_PS'][:Nbr_test]
h_1R_test = dataset_test['h_PR'][:Nbr_test]
h_21_test = dataset_test['h_SP'][:Nbr_test]
h_22_test = dataset_test['h_SS'][:Nbr_test]
h_2R_test = dataset_test['h_SR'][:Nbr_test]
h_R1_test = dataset_test['h_RP'][:Nbr_test]
h_R2_test = dataset_test['h_RS'][:Nbr_test]

x_test = np.stack([h_R1_test, h_11_test, h_2R_test, h_1R_test, h_22_test, h_R2_test, h_21_test, h_12_test], axis=1)



## Application of the closed-form solution to imperfect CSI and perfect CSI

In [14]:
# Get Noisy H-MATRIX with different noise variance
try : 
    noise_levels, imperfect_csi_matrix = noise_to_channels(x_test, Primary_ID, Secondary_ID)

    # Compute bruteforce for NH-MATRIX

    imperfect_csi_results = AS_for_noisy_channels_CF(imperfect_csi_matrix) 

    # bruteforce for test_set without noise

    perfect_csi_results = generate_benchmark_CF(x_test) 

    #----------------------------------Save data----------------------------------------------#

    imperfect_csi_matrix_path = data_path+'/imperfect_csi_matrix_CF'
    np.savez(imperfect_csi_matrix_path, imperfect_csi_matrix)


    imperfect_csi_results_path = data_path+'/imperfect_csi_results_CF'
    np.savez(imperfect_csi_results_path, imperfect_csi_results)


    perfect_csi_results_path = data_path+'/perfect_csi_results_CF'
    np.savez(perfect_csi_results_path, perfect_csi_results)

except FileExistsError : 
    pass

## loading the data 

In [15]:
# First, load ... 
# useful for DNN prediction because DNN takes H as input 

imperfect_csi_matrix_path = data_path+'/imperfect_csi_matrix_CF'

imperfect_csi_matrix = np.load(imperfect_csi_matrix_path+".npz")
imperfect_csi_matrix.files
imperfect_csi_matrix = imperfect_csi_matrix['arr_0']

# Noisy_H ==> NH_MATRIX

#-----------------------------------------------------------------------------------------#
# useful for Rate, outage, Delta calculation ...  because it containing (Alpha, Pr, Ps) 


imperfect_csi_results_path = data_path+'/imperfect_csi_results_CF'

imperfect_csi_results = np.load(imperfect_csi_results_path+".npz")
imperfect_csi_results.files
imperfect_csi_results = imperfect_csi_results['arr_0']

#-----------------------------------------------------------------------------------------#
# useful for Rate, outage, Delta calculation, because it containing G Matrix without noise 


perfect_csi_results_path = data_path+'/perfect_csi_results_CF'

perfect_csi_results = np.load(perfect_csi_results_path+".npz")
perfect_csi_results.files
perfect_csi_results = perfect_csi_results['arr_0']

## Testing the closed form solution

In [18]:
db_bruteforce = opportunistic_rate_for_noisy_channels_CF(perfect_csi_results, imperfect_csi_results) 
# rate for H without noise
db_benchmark = C(f_obj(perfect_csi_results[:,0], perfect_csi_results[:,1], perfect_csi_results[:,2], perfect_csi_results[:,3], perfect_csi_results[:,4], perfect_csi_results[:,5], perfect_csi_results[:,6], perfect_csi_results[:,7], perfect_csi_results[:,8], perfect_csi_results[:,9]))

# List of relative gap between each noisy matrix and benchmark (BF results without noise)

db_gap = np.array([relative_avreage_gap(db_bruteforce[0,:], db_benchmark[:]),\
                   relative_avreage_gap(db_bruteforce[1,:], db_benchmark[:]),\
                   relative_avreage_gap(db_bruteforce[2,:], db_benchmark[:]),\
                   relative_avreage_gap(db_bruteforce[3,:], db_benchmark[:]),\
                   relative_avreage_gap(db_bruteforce[4,:], db_benchmark[:]),\
                   relative_avreage_gap(db_bruteforce[5,:], db_benchmark[:]),\
                   relative_avreage_gap(db_bruteforce[6,:], db_benchmark[:])])

db_gap = db_gap*100

In [25]:
Mean_ARPD, Max_ARPD, Mean_Outage_ARPD, Outage_ARPD = PARD_for_noisy_data_CF(perfect_csi_results, imperfect_csi_results)

In [26]:
try : 
    #----------------------------------Save data----------------------------------------------#
    path = 'stats'
    os.mkdir(path)
    db_gap_path = path+'/db_gap_CF'
    np.savez(db_gap_path, db_gap)
    
    mean_ARDP_path = path+'/mean_ARDP_CF'
    np.savez(mean_ARDP_path, Mean_ARPD)


    Max_ARPD_path = path+'/max_ARPD_CF'
    np.savez(Max_ARPD_path, Max_ARPD)
    
    Mean_Outage_ARPD_path = path+'/mean_outage_ARPD_CF'
    np.savez(Mean_Outage_ARPD_path,  Mean_Outage_ARPD)
    
    Outage_ARPD_path = path+'/outage_ARPD_CF'
    np.savez(Outage_ARPD_path,  Outage_ARPD)


except FileExistsError : 
    pass

## Deep Neural Network (perfect CSI only)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

metrics = [Delta_DNN_CF, Opportunistic_Achievable_Rate_CF(tau),outage_percentage_CF(tau)] #, QoS_mean_DF, QoS_median_DF


model = get_model_CF(x_train, loss_CF(LD,tau), metrics, custom_sigmoid, custom_sigmoid, LR) #lr_v
history = model.fit(x_train, np.power(x_train,2), epochs=Epochs, batch_size=BS, validation_split = VS,callbacks=[callback])

model.save('weights_model/GF_model_CF.h5')
np.save('weights_model/GF_model_CF',history.history)





## Testing DNN with imperfect CSI

In [27]:
model_weight_path = "weights_model/GF_model_CF.h5"

DNN_pred = DNN_predictions_CF(imperfect_csi_matrix, model_weight_path)
db_DNN = opportunistic_rate_for_noisy_channels_DNN_CF(perfect_csi_results, DNN_pred)
db_benchmark = C(f_obj(perfect_csi_results[:,0], perfect_csi_results[:,1], perfect_csi_results[:,2], perfect_csi_results[:,3], perfect_csi_results[:,4], perfect_csi_results[:,5], perfect_csi_results[:,6], perfect_csi_results[:,7], perfect_csi_results[:,8], perfect_csi_results[:,9]))

db_gap_DNN = np.array([relative_avreage_gap(db_DNN[0,:], db_benchmark[:]),\
                       relative_avreage_gap(db_DNN[1,:], db_benchmark[:]),\
                       relative_avreage_gap(db_DNN[2,:], db_benchmark[:]),\
                       relative_avreage_gap(db_DNN[3,:], db_benchmark[:]),\
                       relative_avreage_gap(db_DNN[4,:], db_benchmark[:]),\
                       relative_avreage_gap(db_DNN[5,:], db_benchmark[:]),\
                       relative_avreage_gap(db_DNN[6,:], db_benchmark[:])])

db_gap_DNN = db_gap_DNN*100

In [31]:
Mean_ARPD_DNN, Max_ARPD_DNN, Mean_Outage_ARPD_DNN, Outage_ARPD_DNN = PARD_for_noisy_data_DNN_CF(perfect_csi_results, DNN_pred)


In [32]:
try : 
    #----------------------------------Save data----------------------------------------------#
    path = 'stats'
    
    db_gap_path = path+'/db_gap_DNN_CF'
    np.savez(db_gap_path, db_gap_DNN)
    
    mean_ARDP_path = path+'/mean_ARDP_DNN_CF'
    np.savez(mean_ARDP_path, Mean_ARPD_DNN)


    Max_ARPD_path = path+'/max_ARPD_DNN_CF'
    np.savez(Max_ARPD_path, Max_ARPD_DNN)
    
    Mean_Outage_ARPD_path = path+'/mean_outage_ARPD_DNN_CF'
    np.savez(Mean_Outage_ARPD_path,  Mean_Outage_ARPD_DNN)
    
    Outage_ARPD_path = path+'/outage_ARPD_DNN_CF'
    np.savez(Outage_ARPD_path,  Outage_ARPD_DNN)


except FileExistsError : 
    pass

## DNN (Robust training)

## Load imperfect CSI training set

In [37]:
X_train_noised_path = data_path+'X_train_noised'

X_train_noised = np.load(X_train_noised_path+".npz")
X_train_noised.files
X_train_noised = X_train_noised['arr_0']

## Training DNN with imperfect CSI

In [38]:
metrics = [Delta_DNN_CF, Opportunistic_Achievable_Rate_CF(tau),outage_percentage_CF(tau)] #, QoS_mean_DF, QoS_median_DF

hyperparameters = {'lambda' : LD, 'tau': tau, 'metrics': metrics, 'learning_rate': LR, 'epochs':Epochs, 'batch_size': BS, 'validation_split': VS}

DNN_for_noisy_channels_CF(x_train[:Nbr_train,:], X_train_noised[:,:Nbr_train,:], hyperparameters)


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_42 (Dense)                (None, 128)          1152        input_8[0][0]                    
__________________________________________________________________________________________________
dense_43 (Dense)                (None, 256)          33024       dense_42[0][0]                   
__________________________________________________________________________________________________
dense_44 (Dense)                (None, 256)          65792       dense_43[0][0]                   
____________________________________________________________________________________________

In [40]:
DNN_pred = noised_DNN_pred_CF(imperfect_csi_matrix, 'weights_model/N_DNN_CF_')

In [48]:

loss_history, val_loss_history = plot_history_noised_DNN('weights_model/N_DNN_CF_') 


In [49]:
db_N_DNN = opportunistic_rate_for_noisy_channels_DNN(perfect_csi_results, DNN_pred)
db_benchmark = C(f_obj(perfect_csi_results[:,0], perfect_csi_results[:,1], perfect_csi_results[:,2], perfect_csi_results[:,3], perfect_csi_results[:,4], perfect_csi_results[:,5], perfect_csi_results[:,6], perfect_csi_results[:,7], perfect_csi_results[:,8], perfect_csi_results[:,9]))

Mean_ARPD_N_DNN, Max_ARPD_N_DNN, Mean_Outage_ARPD_N_DNN, Outage_ARPD_N_DNN = PARD_for_noisy_data_DNN(perfect_csi_results, DNN_pred)



In [50]:
X = {"opportunistic_rate":db_N_DNN, "mean_ARPD":Mean_ARPD_N_DNN,"max_ARPD":Max_ARPD_N_DNN,\
    "mean_outage":Mean_Outage_ARPD_N_DNN, "outage_ARPD":Outage_ARPD_N_DNN}

rate_gap_dnn, mean_pard_dnn, max_pard_dnn, mean_outage_pard_dnn, outage_pard_dnn = get_dnn_results(X, db_benchmark)

In [51]:
try : 
    #----------------------------------Save data----------------------------------------------#
    path = 'stats'
    
    db_gap_path = path+'/db_gap_N_DNN_CF'
    np.savez(db_gap_path, rate_gap_dnn)
    
    mean_ARDP_path = path+'/mean_ARDP_N_DNN_CF'
    np.savez(mean_ARDP_path, mean_pard_dnn)


    Max_ARPD_path = path+'/max_ARPD_N_DNN_CF'
    np.savez(Max_ARPD_path, max_pard_dnn)
    
    Mean_Outage_ARPD_path = path+'/mean_outage_ARPD_N_DNN_CF'
    np.savez(Mean_Outage_ARPD_path,  mean_outage_pard_dnn)
    
    Outage_ARPD_path = path+'/outage_ARPD_N_DNN_CF'
    np.savez(Outage_ARPD_path,  outage_pard_dnn)


except FileExistsError : 
    pass